# Effective types

In [ ]:
//%cflags:-Wall -Werror -std=c17
#include <stdio.h>

unsigned char A[sizeof(unsigned)] = {9};
unsigned* p = (unsigned*) A;

printf("value %u\n", *p); // error?

value 9


# Alignment

In [ ]:
//%cflags: -I. -std=c17
// #include "c23-fallback.h"
#include <stdio.h>
#include <stdalign.h>
#include <inttypes.h>
#include <complex.h>
// #include "crash.h"

// void enable_alignment_check(void);
typedef complex double cdbl;

int main(void) {
  // enable_alignment_check();
  __asm__("pushf\n"
          "orl $0x40000, (%rsp)\n"
          "popf");

  /* An overlay of complex values and bytes. */
  union {
    cdbl val[2];
    unsigned char buf[sizeof(cdbl[2])];
  } toocomplex = {
    .val = { 0.5 + 0.5*I, 0.75 + 0.75*I, },
  };
  printf("size/alignment: %zu/%zu\n",
         sizeof(cdbl), alignof(cdbl));
  /* Run over all offsets, and crash on misalignment. */
  for (size_t offset = sizeof(cdbl); offset; offset /=2) {
    printf("offset\t%zu:\t", offset);
    fflush(stdout);
    cdbl* bp = (cdbl*)(&toocomplex.buf[offset]); // align!
    printf("%g\t+%gI\t", creal(*bp), cimag(*bp));
    fflush(stdout);
    *bp *= *bp;
    printf("%g\t+%gI", creal(*bp), cimag(*bp));
    fputc('\n', stdout);
  }
}


[C kernel] Executable exited with code -7

In [2]:
__asm__("pushf\n"
        "orl $0x40000, (%rsp)\n"
        "popf");

In [1]:
#include <stdlib.h>

char *p;

__asm__("pushf\n"
        "orl $0x40000, (%rsp)\n"
        "popf");

/*
 * malloc() always provides aligned memory.
 * Do not use stack variable like a[9], depending on the compiler you use,
 * a may not be aligned properly.
 */
p = malloc(sizeof(int) + 1);
memset(p, 0, sizeof(int) + 1);

/* making p unaligned */
p++;

printf("%d\n", *(int *)p);

[C kernel] Executable exited with code -7

In [ ]:
// signal(7) — Linux manual page
// https://man7.org/linux/man-pages/man7/signal.7.html

In [3]:
#include <stdlib.h>

char *p;

/*
 * malloc() always provides aligned memory.
 * Do not use stack variable like a[9], depending on the compiler you use,
 * a may not be aligned properly.
 */
p = malloc(sizeof(int) + 1);
memset(p, 0, sizeof(int) + 1);

/* making p unaligned */
p++;

printf("%d\n", *(int *)p);

0


In [5]:
system("man 7 signal");

SIGNAL(7)           Linux Programmer's Manual          SIGNAL(7)

NAME
       signal - overview of signals

DESCRIPTION
       Linux  supports  both POSIX reliable signals (hereinafter
       "standard signals") and POSIX real-time signals.

   Signal dispositions
       Each signal has a current disposition,  which  determines
       how the process behaves when it is delivered the signal.

       The  entries  in  the  "Action" column of the table below
       specify the default disposition for each signal, as  fol‐
       lows:

       Term   Default action is to terminate the process.

       Ign    Default action is to ignore the signal.

       Core   Default  action  is  to  terminate the process and
              dump core (see core(5)).

       Stop   Default action is to stop the process.

       Cont   Default action is to continue the process if it is
              currently stopped.

       A  process  can  change the disposition of a signal using
       sigaction(2) or s

  table wider than line width
  table wider than line width


aved information includes:

             + the program counter register (i.e.,  the  address
               of  the next instruction in the main program that
               should be executed when the  signal  handler  re‐
               turns);

             + architecture-specific register state required for
               resuming the interrupted program;

             + the thread's current signal mask;

             + the thread's alternate signal stack settings.

             (If the signal  handler  was  installed  using  the
             sigaction(2) SA_SIGINFO flag, then the above infor‐
             mation is accessible via the ucontext_t object that
             is  pointed  to by the third argument of the signal
             handler.)

          d) Any signals specified in act->sa_mask  when  regis‐
             tering the handler with sigprocmask(2) are added to
             the thread's signal mask.  The signal being  deliv‐
             ered  is  also  added  to  the  si